In [1]:
import sqlite3
import pandas as pd
import os
# os.chdir(os.path.dirname(r"{}".format(str(os.path.abspath(__file__)))))

# Create a SQL connection to our SQLite database
con = sqlite3.connect("DATABASES/project.db")

cur = con.cursor()

In [2]:

# # Return all results of query
# cur.execute('SELECT plot_id FROM plots WHERE plot_type="Control"')
# cur.fetchall()

# Load the data into a DataFrame
regex_df = pd.read_sql_query("SELECT * from classification_1_regex", con)
match_df = pd.read_sql_query("SELECT * from classification_1_psittaciformes_or_no", con)
link_df = pd.read_sql_query("SELECT * from classification_1_reg_map_match", con)
bird_or_no_df = pd.read_sql_query("SELECT * from classification_1_parse_bird_or_no", con)

match_df.index = match_df.id
regex_df.index = regex_df.id

print(match_df.head(10))
print(regex_df.head(10))
# # Write the new DataFrame to a new SQLite table
# surveys2002.to_sql("surveys2002", con, if_exists="replace")

    id      ad_id  match_cites_parrot  match_common_parrot  \
id                                                           
1    1  552731_us                   0                    1   
2    2  552645_us                   0                    1   
3    3  552633_us                   0                    1   
4    4  552632_us                   0                    1   
5    5  552631_us                   0                    1   
6    6  552630_us                   0                    1   
7    7  552629_us                   0                    1   
8    8  552628_us                   0                    1   
9    9  552627_us                   0                    1   
10  10  552626_us                   0                    1   

                         mapping_match  
id                                      
1       10;125;270;271;272;278;298;332  
2       10;125;270;271;272;278;298;332  
3                   10;115;128;278;355  
4             8;10;105;115;165;278;355  
5        

In [3]:
match_df["word"]=match_df["mapping_match"]\
    .apply(lambda x : ";".join([regex_df.at[int(_),"word"] for _ in x.split(";") if (len(x)>0)]))


In [4]:
words_df=match_df["word"].str.get_dummies(sep=";").add_prefix("word_")
words_df

,word_(amazona),word_African,word_Amazon,word_Amazona,word_Amazone,word_Ara,word_Aratinga,word_Azul,word_Black-cheeked,word_Black-collared,...,word_petit,word_raja,word_red-faced,word_rouge,word_southern,word_tête,word_verde,word_versicolor,word_vert,word_yaco
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25942,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25944,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
words_df=match_df[["id","ad_id"]].join(words_df)
words_df.reset_index(drop=True, inplace=True)

In [6]:
words_df.to_sql("classification_1_analysis", con, if_exists="replace")

In [7]:
# Be sure to close the connection
con.close()